In [4]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
# import xgboost as xgb
# import lightgbm as lgb
# import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
# from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

### step9
* self train take others (fname,softmax) CSV

In [58]:
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,9.922859e-01,1.160772e-09,3.179634e-05,5.715786e-07,2.640641e-07,1.128962e-10,1.293161e-03,6.370284e-06,1.038877e-06,2.052373e-06,...,7.841393e-04,9.618410e-08,4.481183e-08,2.961526e-07,9.081892e-09,2.962598e-08,3.857372e-09,3.031643e-06,1.416673e-05,6.641925e-04
1,3.002034e-03,2.965694e-04,2.977726e-02,1.679675e-04,1.063804e-03,8.306239e-02,7.997359e-05,2.874326e-04,1.202749e-02,1.209377e-02,...,6.073403e-03,1.951586e-01,7.077498e-03,7.520564e-05,4.427043e-02,1.473820e-01,4.340039e-05,7.831454e-04,8.344786e-03,1.864192e-04
2,2.494484e-08,1.174843e-05,5.017233e-07,2.351012e-11,9.999777e-01,3.681796e-11,1.115583e-08,3.243680e-08,1.531619e-11,5.099320e-11,...,1.653997e-09,3.594207e-09,1.690576e-10,1.134693e-08,1.840063e-10,2.223503e-08,6.283718e-11,9.059158e-07,1.058770e-08,7.890440e-14
3,3.323878e-06,5.823746e-02,1.018949e-04,1.861539e-05,1.367080e-04,1.812969e-05,1.701656e-05,7.980833e-01,1.063154e-06,8.958887e-07,...,2.586592e-05,3.468834e-06,4.935263e-04,1.586177e-03,3.014827e-05,7.696834e-06,7.246422e-05,1.641735e-04,8.239783e-04,2.181476e-05
4,9.888434e-16,1.126319e-19,2.553080e-17,8.321930e-19,7.312300e-17,5.402099e-13,3.663156e-10,5.502193e-22,9.999807e-01,2.035391e-10,...,1.694029e-17,7.710019e-18,8.483338e-21,7.118341e-19,1.325608e-15,1.062173e-10,3.339272e-12,1.496960e-12,3.008581e-19,1.119163e-12
5,5.272283e-06,5.950751e-11,2.119299e-10,1.254201e-12,5.307900e-08,5.865075e-12,3.520478e-04,7.972521e-13,8.238658e-06,9.964581e-01,...,8.573247e-08,1.718718e-09,6.746286e-15,5.334774e-09,3.699463e-10,4.674145e-05,1.342211e-08,2.731146e-09,3.781765e-10,3.459954e-09
6,1.130390e-05,2.497085e-06,7.158921e-07,1.458871e-07,9.695534e-08,1.313445e-06,7.158697e-02,1.127371e-08,9.168379e-09,1.663792e-09,...,6.427558e-06,1.109630e-08,1.136870e-07,6.249818e-07,6.903968e-08,1.248389e-06,4.007128e-08,1.610544e-06,6.162454e-07,1.193181e-07
7,6.541553e-13,1.244978e-15,4.571586e-13,1.674230e-13,3.161664e-14,8.623761e-12,1.363458e-07,2.589566e-15,9.997690e-01,3.186544e-08,...,8.734869e-13,1.074673e-14,1.912451e-15,2.237113e-13,1.856531e-10,8.537736e-09,1.287981e-09,2.525349e-09,5.317167e-14,3.141135e-08
8,4.567375e-14,9.449961e-10,2.202703e-10,1.631202e-15,1.870290e-14,1.287883e-16,1.000732e-10,2.189844e-16,2.145838e-13,4.019590e-08,...,9.568423e-12,4.329536e-14,2.394415e-14,2.255417e-13,5.050880e-11,1.445700e-11,4.330598e-12,8.385990e-11,2.367651e-12,2.805691e-20
9,4.003818e-14,3.629902e-16,2.615628e-16,2.358826e-16,1.042092e-11,1.755574e-16,1.335557e-10,1.055733e-16,8.900560e-11,7.206430e-09,...,8.894874e-11,1.671239e-20,1.032263e-20,1.884851e-15,5.799928e-15,2.417266e-09,6.845568e-12,4.079267e-14,1.122774e-14,5.295928e-14


In [55]:
acc_df[un_or_test]

0            leo_resnet_mfcc6_test_1.csv
1            leo_resnet_mfcc6_test_2.csv
2            leo_resnet_mfcc6_test_3.csv
3            leo_resnet_mfcc6_test_4.csv
4            leo_resnet_mfcc6_test_5.csv
5            leo_resnet_mfcc6_test_6.csv
6            leo_resnet_mfcc6_test_7.csv
7            leo_resnet_mfcc6_test_8.csv
8            leo_resnet_mfcc6_test_9.csv
9           leo_resnet_mfcc6_test_10.csv
10           leo_resnet_mfcc7_test_1.csv
11           leo_resnet_mfcc7_test_2.csv
12           leo_resnet_mfcc7_test_3.csv
13           leo_resnet_mfcc7_test_4.csv
14           leo_resnet_mfcc7_test_5.csv
15           leo_resnet_mfcc7_test_6.csv
16           leo_resnet_mfcc7_test_7.csv
17           leo_resnet_mfcc7_test_8.csv
18           leo_resnet_mfcc7_test_9.csv
19          leo_resnet_mfcc7_test_10.csv
20     jerry_resenet152_mfcc3_semi_1.csv
21     jerry_resenet152_mfcc3_semi_2.csv
22     jerry_resenet152_mfcc3_semi_3.csv
23     jerry_resenet152_mfcc3_semi_4.csv
24     jerry_res

In [52]:
csv

'jerry_resnet152_mfcc3_semi_8.csv'

In [56]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test
phase = 'stage_share'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/valid_acc.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
    df.sort_values("fname", inplace=True)  #SEMI: uN + TEST
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod
    print(csv,len(df))
    if i==0:
        train_X = df.values*ratio
    else:
        try:
            train_X += df.values*ratio 
        except:
            train_X += df.values[5763:]*ratio
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))

leo_resnet_mfcc6_test_9.csv 9400
mike_resnet152_4.csv 15163
mike_resnet2_refine_self_6.csv 9400
mike_resnet18_semi_1.csv 15163
jerry_resnet152_mfcc3_semi_8.csv 15163
mike_resnet18_semi_3.csv 15163
mow_mfcc4_resnet152_phase3_test_4.csv 9400
mike_resnet18_semi_7.csv 15163
jerry_resnet152_mfcc3_semi_5.csv 15163
mike_resnet_gen_self_9.csv 9400
mow_mfcc4_resnet152_phase3_test_9.csv 9400
mike_resnet18_semi_8.csv 15163
leo_resnet_mfcc6_test_10.csv 9400
jerry_resnet152_mfcc3_semi_4.csv 15163
mow_mfcc4_resnet18_phase3_test_10.csv 9400
jerry_resnet34_mfcc3_phase3test_5.csv 9400
jerry_resnet34_mfcc3_phase3test_1.csv 9400
jerry_resnet152_mfcc3_semi_6.csv 15163
jerry_resnet152_mfcc3_semi_9.csv 15163
mike_resnet_gen_self_7.csv 9400
mike_resnet_gen_self_10.csv 9400
leo_resnet_mfcc7_test_1.csv 9400
leo_resnet_mfcc7_test_6.csv 9400
leo_resnet_mfcc6_test_5.csv 9400
mike_resnet18_semi_4.csv 15163
leo_resnet_mfcc6_test_8.csv 9400
mike_resnet_gen_self_4.csv 9400
leo_resnet_mfcc7_test_7.csv 9400
leo_resnet_

In [60]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test['label'] = 'none'
# fname = fname_test # 記得註解掉

# un + test
fname_un = pd.read_csv('data/train_label.csv')
fname_un = fname_un[fname_un.manually_verified==0]
fname_un = fname_un.drop(['manually_verified'],axis=1)
fname = fname_un.append(fname_test)
fname.sort_values('fname',inplace=True)
fname.reset_index(drop=True,inplace=True)

fname['label_verified'] = "none"
fname['verified_confidence']=0.0
print(len(fname))
for i,r in fname.iterrows():
    fname.iloc[i,2] = np.argmax(train_X[i])
    fname.iloc[i,3] = max(train_X[i])

df = fname
df

9400


,fname,label,label_verified,verified_confidence
0,00063640.wav,none,30,0.587089
1,0013a1db.wav,none,16,0.817858
2,002bb878.wav,none,21,0.465258
3,002d392d.wav,none,21,0.920451
4,00326aa9.wav,none,15,0.971639
5,0038a046.wav,none,21,0.624841
6,003995fa.wav,none,23,0.266830
7,005ae625.wav,none,38,0.646749
8,007759c4.wav,none,7,0.215247
9,008afd93.wav,none,1,0.974385


In [35]:
dicts_ch = pickle.load(open('data/map_reverse.pkl','rb'))
df['label_verified'] = df['label_verified'].map(dicts_ch)
threshold_upper = df['verified_confidence'].mean() + df['verified_confidence'].std()
threshold_lower = min([df['verified_confidence'].mean() , 0.7])
print("threshold upper , lower:",threshold_upper , threshold_lower)
print("mean , std:",df['verified_confidence'].mean() , df['verified_confidence'].std())
df['take']=0

for i,r in df.iterrows():
    if r['verified_confidence']>threshold_upper:
        df.iloc[i,4]=1
    elif ((r['label'] == r['label_verified'])and(r['verified_confidence']>threshold_lower)):
        df.iloc[i,4]=1
    else:
        df.iloc[i,4]=0
df

threshold upper , lower: 0.962956929023524 0.6868034051035227
mean , std: 0.6868034051035227 0.27615352392000136


,fname,label,label_verified,verified_confidence,take
0,00044347.wav,Hi-hat,Hi-hat,0.508643,0
1,00063640.wav,none,Shatter,0.524175,0
2,0013a1db.wav,none,Flute,0.846859,0
3,002bb878.wav,none,Bass_drum,0.457352,0
4,002d256b.wav,Trumpet,Bark,0.126812,0
5,002d392d.wav,none,Bass_drum,0.915956,0
6,00326aa9.wav,none,Oboe,0.964103,1
7,0038a046.wav,none,Bass_drum,0.590792,0
8,003995fa.wav,none,Squeak,0.328664,0
9,003b91e8.wav,Cello,Cello,0.838155,1


In [36]:
df_ = df[df['take']==1]
df_ = df_.filter(['fname','label_verified'])
df_.to_csv('data/selftrain/Y_selftrain_ens_verified.csv',index=False) # 儲存train_Y
df_

,fname,label_verified
6,00326aa9.wav,Oboe
9,003b91e8.wav,Cello
13,0075d39c.wav,Keys_jangling
17,008afd93.wav,Saxophone
18,00a161c0.wav,Cello
20,00ad7068.wav,Laughter
28,00c9e799.wav,Oboe
29,00ccf065.wav,Bass_drum
30,00ce569f.wav,Hi-hat
31,00d0ab77.wav,Clarinet


## stage 3 ens csv
### predict weigted acc
* original softmax ens

In [67]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test
phase = 'phase4'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/weight_accF.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
#     df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
        try:
            train_X += df.values*ratio 
        except:
            train_X += df.values[5763:]*ratio
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(9400, 41)
0.9999999878836178


In [68]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test['label'] = 'none'
fname = fname_test # 記得註解掉如果是un+test
'''
fname_un = pd.read_csv('data/train_label.csv')
fname_un = fname_un[fname_un.manually_verified==0]
fname_un = fname_un.drop(['manually_verified'],axis=1)
fname = fname_un.append(fname_test)
fname.sort_values('fname',inplace=True)
fname.reset_index(drop=True,inplace=True)
'''
fname['label_verified'] = "none"
fname['verified_confidence']=0.0
print(len(fname))
for i,r in fname.iterrows():
    top3 = train_X[i].argsort()[-3:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)
    fname.iloc[i,2] = s#np.argmax(train_X[i])
    fname.iloc[i,3] = max(train_X[i])

df = fname
print(df['verified_confidence'].min(),df['verified_confidence'].mean(), df['verified_confidence'].std() )
df

9400
0.08217809405680537 0.7515160645699981 0.26395262115225393


,fname,label,label_verified,verified_confidence
0,00063640.wav,none,Shatter Keys_jangling Tambourine,0.595603
1,0013a1db.wav,none,Flute Trumpet Violin_or_fiddle,0.823982
2,002bb878.wav,none,Bass_drum Knock Gunshot_or_gunfire,0.491480
3,002d392d.wav,none,Bass_drum Knock Double_bass,0.935640
4,00326aa9.wav,none,Oboe Clarinet Bass_drum,0.975470
5,0038a046.wav,none,Bass_drum Electric_piano Knock,0.671415
6,003995fa.wav,none,Squeak Clarinet Telephone,0.260038
7,005ae625.wav,none,Acoustic_guitar Cello Clarinet,0.660926
8,007759c4.wav,none,Clarinet Flute Telephone,0.223711
9,008afd93.wav,none,Saxophone Flute Cello,0.976477


In [69]:
df_fin = df[df.label=='none']
df_fin = df_fin.filter(['fname','label_verified'])
df_fin.columns = ['fname','label']
df_fin.to_csv('result/final_ens_stage3.csv',index=False)
df_fin

,fname,label
0,00063640.wav,Shatter Keys_jangling Tambourine
1,0013a1db.wav,Flute Trumpet Violin_or_fiddle
2,002bb878.wav,Bass_drum Knock Gunshot_or_gunfire
3,002d392d.wav,Bass_drum Knock Double_bass
4,00326aa9.wav,Oboe Clarinet Bass_drum
5,0038a046.wav,Bass_drum Electric_piano Knock
6,003995fa.wav,Squeak Clarinet Telephone
7,005ae625.wav,Acoustic_guitar Cello Clarinet
8,007759c4.wav,Clarinet Flute Telephone
9,008afd93.wav,Saxophone Flute Cello


In [83]:
df_ens = pd.DataFrame(df_fin.label.str.split(' ',2).tolist(),columns=['1','2','3'])
df_ens = pd.merge(pd.DataFrame(df_fin.fname),df_ens,how='inner',right_index=True,left_index=True)
df_ens['lp']='none'
df_ens

,fname,1,2,3
0,00063640.wav,Shatter,Keys_jangling,Tambourine
1,0013a1db.wav,Flute,Trumpet,Violin_or_fiddle
2,002bb878.wav,Bass_drum,Knock,Gunshot_or_gunfire
3,002d392d.wav,Bass_drum,Knock,Double_bass
4,00326aa9.wav,Oboe,Clarinet,Bass_drum
5,0038a046.wav,Bass_drum,Electric_piano,Knock
6,003995fa.wav,Squeak,Clarinet,Telephone
7,005ae625.wav,Acoustic_guitar,Cello,Clarinet
8,007759c4.wav,Clarinet,Flute,Telephone
9,008afd93.wav,Saxophone,Flute,Cello


## stage1 label spreading

In [70]:
folder = 'data/stacking/lp_model_res/' 
files = os.listdir(folder)
files

['Y_test_denseLGD_fold_stacking_410_dense2.npy',
 'Y_test_denseLGD_fold_stacking_410_resnet.npy',
 'Y_test_denseLGD_fold_stacking_410_dense4.npy',
 'Y_test_denseLGD_fold_stacking_41_densenet.npy',
 'Y_test_denseLGD_fold_stacking_410_dense1.npy',
 'Y_test_denseLGD_fold_stacking_410_dense3.npy',
 'Y_test_denseLGD_fold_stacking_41_densenet2.npy',
 'Y_test_denseLGD_fold_stacking_41_only1.npy',
 'Y_test_denseLGD_fold_stacking_ori_resnet.npy',
 'Y_test_denseLGD_fold_stacking_410_only2.npy',
 'Y_test_denseLGD_fold_stacking_410_resnet2.npy',
 'Y_test_denseLGD_fold_stacking_410_only1.npy']

In [72]:
df_un_ans = pd.DataFrame()
for fold in files:
    un_ans = np.load(folder+fold)
    df = pd.DataFrame(un_ans).T
    df_un_ans = df_un_ans.append(df)
col_list = []
for col_num in range(len(df_un_ans.columns)):

    counter = df_un_ans[col_num].value_counts()

    col_list.append(dict(counter))

len(col_list) 

9400

In [76]:
take_list=[]
take_label=[]
for i,stats in enumerate(col_list):
    if max(stats.values()) >=len(files):
        take_list.append(i)
        ens_label = max(stats.keys(), key=(lambda k: stats[k]))
        take_label.append(ens_label)
print(len(take_list))

4523


In [87]:
for i,row in enumerate(take_list):
    df_ens.iloc[row,4]=take_label[i]
df_ens['lp']=df_ens.lp.map(reverse_dict)
df_ens

,fname,1,2,3,lp
0,00063640.wav,Shatter,Keys_jangling,Tambourine,NaN
1,0013a1db.wav,Flute,Trumpet,Violin_or_fiddle,Flute
2,002bb878.wav,Bass_drum,Knock,Gunshot_or_gunfire,NaN
3,002d392d.wav,Bass_drum,Knock,Double_bass,Bass_drum
4,00326aa9.wav,Oboe,Clarinet,Bass_drum,NaN
5,0038a046.wav,Bass_drum,Electric_piano,Knock,NaN
6,003995fa.wav,Squeak,Clarinet,Telephone,NaN
7,005ae625.wav,Acoustic_guitar,Cello,Clarinet,NaN
8,007759c4.wav,Clarinet,Flute,Telephone,NaN
9,008afd93.wav,Saxophone,Flute,Cello,Saxophone


## stage 2 stacking model

In [88]:
folder = 'data/stacking/nn/' 
files = os.listdir(folder)
files

['LGD_fold_stacking_410_resnet2.npy',
 'LGD_fold_stacking_410_dense2.npy',
 'LGD_fold_stacking_410_only1.npy',
 'LGD_fold_stacking_410_only2.npy',
 'LGD_fold_stacking_410_dense1.npy',
 'LGD_fold_stacking_410_resnet.npy',
 'LGD_fold_stacking_41_densenet.npy',
 'LGD_fold_stacking_41_densenet2.npy',
 'LGD_fold_stacking_41_only1.npy',
 'LGD_fold_stacking_410_dense4.npy',
 'LGD_fold_stacking_410_dense3.npy']

In [91]:
un_or_test = 'stack'
acc_df = pd.read_csv('data/stacking/stack_accF.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
# acc_df = acc_df.filter([un_or_test,'acc'])

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = np.load(folder+csv)#pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
#     df.sort_values("fname", inplace=True)
#     if df.iloc[0,0] == 'fname':
#         df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
#     df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df*ratio
    else:
        train_X += df*ratio 
#         except:
#             train_X += df.values[5763:]*ratio
print(train_X.shape)
# train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(9400, 41)
33.5878803730011


In [95]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test['label'] = 'none'
for i,r in fname_test.iterrows():
    top3 = train_X[i].argsort()[-1:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)
    fname_test.iloc[i,1] = s#np.argmax(train_X[i])
#     fname.iloc[i,3] = max(train_X[i])
fname_test.columns = ['fname','stack']
fname_test

,fname,stack
0,00063640.wav,Shatter
1,0013a1db.wav,Flute
2,002bb878.wav,Bass_drum
3,002d392d.wav,Bass_drum
4,00326aa9.wav,Oboe
5,0038a046.wav,Bass_drum
6,003995fa.wav,Squeak
7,005ae625.wav,Acoustic_guitar
8,007759c4.wav,Flute
9,008afd93.wav,Saxophone


In [96]:
df_all = pd.merge(df_ens,fname_test,how='inner',on='fname')
df_all

,fname,1,2,3,lp,stack
0,00063640.wav,Shatter,Keys_jangling,Tambourine,NaN,Shatter
1,0013a1db.wav,Flute,Trumpet,Violin_or_fiddle,Flute,Flute
2,002bb878.wav,Bass_drum,Knock,Gunshot_or_gunfire,NaN,Bass_drum
3,002d392d.wav,Bass_drum,Knock,Double_bass,Bass_drum,Bass_drum
4,00326aa9.wav,Oboe,Clarinet,Bass_drum,NaN,Oboe
5,0038a046.wav,Bass_drum,Electric_piano,Knock,NaN,Bass_drum
6,003995fa.wav,Squeak,Clarinet,Telephone,NaN,Squeak
7,005ae625.wav,Acoustic_guitar,Cello,Clarinet,NaN,Acoustic_guitar
8,007759c4.wav,Clarinet,Flute,Telephone,NaN,Flute
9,008afd93.wav,Saxophone,Flute,Cello,Saxophone,Saxophone


In [103]:
def one(x):
    if x['lp'] != 'none':
        return x['lp']
    else:
        return x['stack']
def two(x):
    if x['fin1'] == x['stack'] == x['1']:
        return x['2']
    elif x['fin1'] == x['stack']:
        return x['1']
    else:
        return x['stack']
def three(x):
    if x['fin2'] == x['1']:
        return x['2']
    elif x['fin2'] == x['2']:
        return x['3']
    else:
        return x['1']

In [108]:
1!=2!=3

True

In [115]:
def one(x):
    if x['lp'] == x['stack']:
        return x['lp']
    elif x['lp'] == x['1'] :
        return x['lp']
    elif x['stack'] ==x['1']:
        return x['stack']
    elif x['lp'] != 'none':
        return x['lp']
    else:
        return x['stack']
def two(x):
    if x['fin1'] != x['1']:
        return x['1']
    elif x['fin1'] != x['2']:
        return x['2']
    else:
        return x['3']
def three(x):
    if x['fin2'] == x['3']:
        if x['lp'] != 'none':
            return x['lp']
        else :
            return x['stack']
    elif x['fin2'] != x['2']:
        return x['2']
    else:
        return x['3']

In [116]:
df_all['fin1'] = df_all.apply(one,axis=1)
df_all['fin2'] = df_all.apply(two,axis=1)
df_all['fin3'] = df_all.apply(three,axis=1)
df_all['final'] = df_all['fin1']+' '+df_all['fin2']+' '+df_all['fin3']
df_all

,fname,1,2,3,lp,stack,fin1,fin2,fin3,final
0,00063640.wav,Shatter,Keys_jangling,Tambourine,none,Shatter,Shatter,Keys_jangling,Tambourine,Shatter Keys_jangling Tambourine
1,0013a1db.wav,Flute,Trumpet,Violin_or_fiddle,Flute,Flute,Flute,Trumpet,Violin_or_fiddle,Flute Trumpet Violin_or_fiddle
2,002bb878.wav,Bass_drum,Knock,Gunshot_or_gunfire,none,Bass_drum,Bass_drum,Knock,Gunshot_or_gunfire,Bass_drum Knock Gunshot_or_gunfire
3,002d392d.wav,Bass_drum,Knock,Double_bass,Bass_drum,Bass_drum,Bass_drum,Knock,Double_bass,Bass_drum Knock Double_bass
4,00326aa9.wav,Oboe,Clarinet,Bass_drum,none,Oboe,Oboe,Clarinet,Bass_drum,Oboe Clarinet Bass_drum
5,0038a046.wav,Bass_drum,Electric_piano,Knock,none,Bass_drum,Bass_drum,Electric_piano,Knock,Bass_drum Electric_piano Knock
6,003995fa.wav,Squeak,Clarinet,Telephone,none,Squeak,Squeak,Clarinet,Telephone,Squeak Clarinet Telephone
7,005ae625.wav,Acoustic_guitar,Cello,Clarinet,none,Acoustic_guitar,Acoustic_guitar,Cello,Clarinet,Acoustic_guitar Cello Clarinet
8,007759c4.wav,Clarinet,Flute,Telephone,none,Flute,Flute,Clarinet,Flute,Flute Clarinet Flute
9,008afd93.wav,Saxophone,Flute,Cello,Saxophone,Saxophone,Saxophone,Flute,Cello,Saxophone Flute Cello


In [117]:
dfF = df_all.filter(['fname','final'])
dfF.columns = ['fname','label']
dfF

,fname,label
0,00063640.wav,Shatter Keys_jangling Tambourine
1,0013a1db.wav,Flute Trumpet Violin_or_fiddle
2,002bb878.wav,Bass_drum Knock Gunshot_or_gunfire
3,002d392d.wav,Bass_drum Knock Double_bass
4,00326aa9.wav,Oboe Clarinet Bass_drum
5,0038a046.wav,Bass_drum Electric_piano Knock
6,003995fa.wav,Squeak Clarinet Telephone
7,005ae625.wav,Acoustic_guitar Cello Clarinet
8,007759c4.wav,Flute Clarinet Flute
9,008afd93.wav,Saxophone Flute Cello


In [105]:
dfF = df_all.filter(['fname','final'])
dfF.columns = ['fname','label']
dfF

,fname,label
0,00063640.wav,Shatter Keys_jangling Tambourine
1,0013a1db.wav,Flute Trumpet Violin_or_fiddle
2,002bb878.wav,Bass_drum Knock Gunshot_or_gunfire
3,002d392d.wav,Bass_drum Knock Double_bass
4,00326aa9.wav,Oboe Clarinet Bass_drum
5,0038a046.wav,Bass_drum Electric_piano Knock
6,003995fa.wav,Squeak Clarinet Telephone
7,005ae625.wav,Acoustic_guitar Cello Clarinet
8,007759c4.wav,Flute Clarinet Flute
9,008afd93.wav,Saxophone Flute Cello


In [118]:
dfF.to_csv('result/LGD_lpANDstack_123.csv',index=False)